Theoretical Questions

Exercise 1:
⊛ OLS is prone to overfitting in high-dimensional datasets. Also, high dimension datasets exhibit multicolinearity. 
⊛ Using regularization methods like ridge regression would be better. This is because adding regularization terms can help prevent overfitting by discouraging large coefficients. 

Exercise 2:
(c) The objective function is the same as ordinary least squares objective function.

Exercise 3: 
(i) (b) and (c)

Exercise 4:
High Bias. Reduce regularization parameter.


# Coding Question

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Exercise 5

# (a)
car_price=pd.read_csv('CarPrice_Assignment.csv')

# (b)
X=car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower',
 'peakrpm', 'citympg', 'highwaympg']]
y=car_price['price']
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2)

# (c)
coefficients_df = pd.DataFrame(columns=X.columns)
for _ in range(1000):
    lasso = Lasso()
    scaler = StandardScaler()
    pipeline = Pipeline([('scaler', scaler), ('lasso', lasso)])
    pipeline.fit(X_train, y_train)
    coefficients_df.loc[len(coefficients_df)] = pipeline.named_steps['lasso'].coef_
zero_counts = (coefficients_df == 0).sum()
variables_to_remove = zero_counts[zero_counts > 500].index.tolist()
X_train = X_train.drop(columns=variables_to_remove)
X_test = X_test.drop(columns=variables_to_remove)

# (d)
linear_reg = LinearRegression()
linear_reg_pipeline = Pipeline([('scaler', StandardScaler()), ('linear_reg', linear_reg)])
linear_reg_pipeline.fit(X_train, y_train)
y_pred_linear = linear_reg_pipeline.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)

# (e)
ridge = Ridge()
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(ridge, param_grid, cv=5)
ridge_pipeline = Pipeline([('scaler', StandardScaler()), ('ridge', grid_search)])
ridge_pipeline.fit(X_train, y_train)
optimal_lambdas = []
for _ in range(100):
    ridge_pipeline.fit(X_train, y_train)
    optimal_lambda = ridge_pipeline.named_steps['ridge'].best_params_['alpha']
    optimal_lambdas.append(optimal_lambda)
most_common_lambda = np.median(optimal_lambdas)

ridge = Ridge(alpha=most_common_lambda)
ridge_pipeline = Pipeline([('scaler', StandardScaler()), ('ridge', ridge)])
ridge_pipeline.fit(X_train, y_train)
y_pred_ridge = ridge_pipeline.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

# (f)
print("Linear Regression MSE:", mse_linear)
print("Ridge Regression MSE:", mse_ridge)
if mse_linear < mse_ridge:
    print("I would use Linear Regression to predict car prices because it has a lower MSE.")
else:
    print("I would use Ridge Regression to predict car prices because it has a lower MSE.")


Linear Regression MSE: 11879466.418280955
Ridge Regression MSE: 12167645.251862932
I would use Linear Regression to predict car prices because it has a lower MSE.
